In [1]:
import os
from typing import Dict, List, Union

from neo4j import GraphDatabase
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

### Neo4j Driver

In [33]:
driver = GraphDatabase.driver(uri=os.environ.get("NEO4J_URI"), auth=(os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD")), liveness_check_timeout=0.0)
driver.verify_authentication()
driver.verify_connectivity()

In [39]:
ravi_query: str = """
// Ravi's
MATCH (n:dbt_table_or_consumption_view {name: 'prod_khc_sales.kroger.kroger_daily_point_of_sale_fact'})
WITH n
MATCH p=()-[*0..]->(n)
WITH n, nodes(p) AS nodes, relationships(p) AS rels
with n, { node: head(nodes) , rel: head(rels) } AS root
WITH n, COLLECT(DISTINCT root) AS roots
MATCH p=(n)-[*0..]->()
WITH n, roots, nodes(p) AS nodes, relationships(p) AS rels
with n, roots, { node: tail(nodes) , rel: CASE WHEN size(rels) > 1
THEN tail(rels) ELSE head(rels) END } AS leaf
WITH {node: n, rel: []} as n1, roots, COLLECT(DISTINCT leaf)
AS leaves
RETURN n1 + roots + leaves as data
"""

updated_query: str = """
MATCH (n:dbt_table_or_consumption_view {name: 'prod_khc_sales.kroger.kroger_daily_point_of_sale_fact'})
// Get Roots
CALL {
    WITH n
    CALL apoc.path.expandConfig(n, {
	    relationshipFilter: "<",
        uniqueness: "NONE",
        minLevel: 0,
        maxLevel: 10
    })
    YIELD path AS p
    WITH nodes(p) AS nodes, 
         relationships(p) AS rels
    WITH { node: head(nodes) , rel: head(rels) } AS root
    WITH COLLECT(DISTINCT root) AS roots
    RETURN roots
}
WITH n, roots
// Get Leafs
CALL {
    WITH n
    CALL apoc.path.expandConfig(n, {
	    relationshipFilter: ">",
        uniqueness: "NONE",
        minLevel: 0,
        maxLevel: 1000
    })
    YIELD path AS p
    WITH nodes(p) AS nodes, 
         relationships(p) AS rels
    WITH { node: head(nodes) , rel: head(rels) } AS leaf
    WITH COLLECT(DISTINCT leaf) AS leaves
    RETURN leaves
}
RETURN n + roots + leaves
"""

In [40]:
def format_query(query: str) -> str:
    query = query.replace("{", "{{")
    return query.replace("}", "}}")

In [41]:
def run_query(query: str) -> List[Union[List[str], str]]:
    with driver.session() as session:
        return session.run(query).values()[0]

In [42]:
res = run_query(updated_query)

In [43]:
len(res[0])

27